# Real Data Experiments - Tree Visualization

## Import Statements

In [12]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), "../src"))

import pandas as pd
import numpy as np
import skbio.stats.composition as cmp
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

import dirichlet
from classo import classo_problem

from method_fct import LinearInstrumentModel
from utils_real import *
import ibis_util_functions as util
import toyplot
import toyplot.pdf, toyplot.png
from visualization import draw_tree

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load in Data

Different aggregation levels:
- Pylum
- Class
- Order
- Family
- Genus
- Species


## Preprocess the Data

In [14]:
agg_level = "Genus"


tax_levels = ["Kingdom", "Phylum", "Class", "Order", "Family", "Genus"]
input_path = "/Users/elisabeth.ailer/Projects/P1_Microbiom/Code/input/data/orgData_Day21_"+agg_level+"_231228.csv"
save_path = "/Users/elisabeth.ailer/Projects/P1_Microbiom/Code/Figures/RealData"

In [15]:
# load real data on different aggregation levels

data = pd.read_csv(input_path, index_col=0)

Y = data.iloc[-1, :].values
X_count = data[data.sum(axis=1)!=0].iloc[:-1, :].T.values

X_count = add_pseudo_count(X_count)
n, p = X_count.shape
num_inst = 1  # number of instruments, here only 1
X = np.array([X_count[i, :]/X_count[i, :].sum() for i in range(n)])
Z = np.array([data.columns.values[i].find("control") for i in range(n)])*(-1)

bac_names = list(data[data.sum(axis=1)!=0].iloc[:-1, :].index)

## Results of `Biostats_RealData_Analysis`

In [16]:
# put results here

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))
 
if agg_level=="Phylum":
    bac_onestage = ["Proteobacteria", "Bacteroidetes", "Actinobacteria"]
    bac_twostage = ["Proteobacteria", "Bacteroidetes"]
    bac_dirlc = ["Proteobacteria", "Cyanobacteria", "Bacteroidetes"]  # be aware that the direction is different than the two stage method
    
if agg_level=="Class":
    bac_onestage = ["Gammaproteobacteria", "Actinobacteria"]
    bac_twostage = ["Bacteroidia", "Bacilli"]
    bac_dirlc = ["Chloroplast", "Verrucomicrobiae", "Bacilli"]

if agg_level=="Order":
    bac_onestage = ["Enterobacteriales", "RF39"]
    bac_twostage = ["Lactobacillales", "Bacteroidales"]
    bac_dirlc = ["Xanthomonadales", "Pseudomonadales"]
    
if agg_level=="Family":
    bac_onestage = ["Enterobacteriaceae", "Lactobacillaceae", "Peptococcaceae", "Clostridiaceae"]
    bac_twostage = ["S24-7", "Enterococcaceae"]
    bac_dirlc = ["Burkholderiaceae", "Erysipelotrichaceae", "Turicibacteraceae"]
    
if agg_level=="Genus":
    bac_onestage = ["Ent.|unclassified", "Lactobacillus"]
    bac_twostage = ["Blautia", "Anaerostipes"]
    bac_dirlc = ["Clostridium", "Candidatus Arthromitus", "Geobacillus"]

bac_agreed = intersection(bac_onestage, bac_twostage)

In [17]:
# setup input for visualization
node_df = util.get_phylo_levels(pd.DataFrame(data=bac_names, columns=["Cell Type"]), agg_level)
node_df.index = bac_names
node_df.loc[(node_df["Family"] == "Enterobacteriaceae")&(node_df["Genus"]==""), "Genus"]="Ent.|unclassified"
node_df["Final Parameter"] = "lightgray"
node_df.loc[node_df[agg_level].isin(bac_onestage), "Final Parameter"]= "blue"
node_df.loc[node_df[agg_level].isin(bac_twostage), "Final Parameter"]= "orange"
node_df.loc[node_df[agg_level].isin(bac_agreed), "Final Parameter"]= "black"
node_df[agg_level].replace('', np.nan, inplace=True)
node_df.dropna(subset=[agg_level], inplace=True)
#node_df = node_df.iloc[: -3, :]
node_df["Cell Type"] = node_df.index

## Visualization of Microbiome Data and Results

In [18]:
# input for visualization
node_df.head()

,Kingdom,Phylum,Class,Order,Family,Genus,Final Parameter,Cell Type
Bacteria*Firmicutes*Erysipelotrichi*Erysipelotrichales*Erysipelotrichaceae*Clostridium,Bacteria,Firmicutes,Erysipelotrichi,Erysipelotrichales,Erysipelotrichaceae,Clostridium,lightgray,Bacteria*Firmicutes*Erysipelotrichi*Erysipelot...
Bacteria*Actinobacteria*Coriobacteriia*Coriobacteriales*Coriobacteriaceae*Adlercreutzia,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Coriobacteriaceae,Adlercreutzia,lightgray,Bacteria*Actinobacteria*Coriobacteriia*Corioba...
Bacteria*Firmicutes*Clostridia*Clostridiales*Clostridiaceae*Candidatus Arthromitus,Bacteria,Firmicutes,Clostridia,Clostridiales,Clostridiaceae,Candidatus Arthromitus,lightgray,Bacteria*Firmicutes*Clostridia*Clostridiales*C...
Bacteria*Proteobacteria*Gammaproteobacteria*Pseudomonadales*Pseudomonadaceae*Pseudomonas,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,lightgray,Bacteria*Proteobacteria*Gammaproteobacteria*Ps...
Bacteria*Proteobacteria*Gammaproteobacteria*Pseudomonadales*Moraxellaceae*Acinetobacter,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,lightgray,Bacteria*Proteobacteria*Gammaproteobacteria*Ps...


In [19]:
tree = draw_tree(node_df, agg_level=agg_level, eff_max=5)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t572bf3789f364968bb1ca45a6bd2fc4d"> Robinsoniella Anaerostipes Roseburia [Ruminococcus] Blautia Dorea Coprococcus Butyrivibrio Oscillospira Anaerotruncus Ruminococcus SMB53 Clostridium CandidatusArthromitus Anaerovorax Tepidimicrobium rc4-4 Dehalobacterium Streptococcus Lactobacillus Enterococcus Anaerobacillus Geobacillus Staphylococcus Turicibacter [Eubacterium] Coprobacillus Allobaculum Clostridium Ent.|unclassified Klebsiella Citrobacter Acinetobacter Pseudomonas Pandoraea Burkholderia Janthinobacterium Sphingomonas Saccharopolyspora Actinomyces Brachybacterium Corynebacterium Bifidobacterium Adlercreutzia Odoribacter Prevotella Elizabethkingia Cloacibacterium Akkermansia Anaeroplasma Phylum Tenericutes Verrucomicrobia Bacteroidetes Actinobacteria Proteobacteria Firmicutes Influential Compositions Only LC ILR+LC Both Methods

In [9]:
# execute this line if you want to save the image
draw_tree(node_df, agg_level=agg_level, eff_max=5, 
          save_path=os.path.join(save_path, "Tree"+str(agg_level)+".svg"))